In [1]:
from pandas_ml_utils import pd


## Fist lets load some data

In [2]:
df = pd.read_csv('SPY.csv')
df.tail()


,Date,Open,High,Low,Close,Adj Close,Volume
6758,2019-11-29,314.859985,315.130005,314.059998,314.309998,314.309998,36592700
6759,2019-12-02,314.589996,314.660004,311.170013,311.640015,311.640015,75767800
6760,2019-12-03,308.649994,309.640015,307.130005,309.549988,309.549988,73941700
6761,2019-12-04,310.700012,312.119995,310.320007,311.459991,311.459991,49080000
6762,2019-12-05,312.230011,312.250000,310.580109,312.089996,312.089996,40781669


## Fit Regularized Model
Now we just build a very naive model where we add an L1 and L2 penalization term. Don't expect the model to predict
anything useful

In [3]:
model_name = '/tmp/regression_with_regularization.model'

# in order to restore from a gobal namespace you need to wrap everything in a function
def train_model():
    with df.model(model_name) as m:
        from pandas_ml_common.utils.column_lagging_utils import lag_columns
        from pandas_ml_utils import FeaturesLabels, RegressionSummary, FittingParameter, FittableModel
        from pandas_ml_utils_torch import PytorchModelProvider
        from torch import nn

        # the network provider needs to be wrapped in a function as well because we can't pickle nested classes
        def net_provider():
            from pandas_ml_utils_torch import PytorchNN
            from torch import nn

            class Net(PytorchNN):

                def __init__(self):
                    super().__init__()
                    self.net = nn.Sequential(
                        nn.Linear(10, 4),
                        nn.Tanh(),
                        nn.Linear(4, 4),
                        nn.Tanh(),
                        nn.Linear(4, 1),
                        nn.Tanh(),
                    )

                def L1(self):
                    # path to the parameters which should be regularized
                    # the path is constructed from self.named_parameters() and allows the use of wildcards
                    return {'net/0/**/weight': 0.02}

                def L2(self):
                    return {
                        'net/0/**/weight': 0.02,
                        'net/2/**/weight': 0.05
                    }

                def forward_training(self, x):
                    return self.net(x)

            return Net()

        def optim(params):
            from torch.optim import SGD
            return SGD(params, lr=0.01, momentum=0.0)

        fit = m.fit(
            FittableModel(
                PytorchModelProvider(
                    net_provider,
                    nn.MSELoss,
                    optim,
                ),
                FeaturesLabels(
                    features=[lambda df: lag_columns(df["Close"].pct_change(), range(10))],
                    labels=[lambda df: df["Close"].pct_change().shift(-1)]
                ),
                summary_provider=RegressionSummary
            ),
            FittingParameter(epochs=100),
            verbose=1
        )

    return fit

fit = train_model()
fit

loss training data: 0.30811557173728943, test data: [0.3081270456314087]
loss training data: 0.3035553991794586, test data: [0.30356425046920776]
loss training data: 0.2991795539855957, test data: [0.2991858124732971]
loss training data: 0.2949829399585724, test data: [0.29498666524887085]
loss training data: 0.2909601330757141, test data: [0.290961354970932]
loss training data: 0.28710564970970154, test data: [0.28710445761680603]
loss training data: 0.28341400623321533, test data: [0.2834104299545288]
loss training data: 0.2798795700073242, test data: [0.27987363934516907]
loss training data: 0.27649664878845215, test data: [0.27648845314979553]
loss training data: 0.27325963973999023, test data: [0.27324923872947693]
loss training data: 0.27016305923461914, test data: [0.27015048265457153]
loss training data: 0.2672012746334076, test data: [0.26718655228614807]
loss training data: 0.26436883211135864, test data: [0.264352023601532]
loss training data: 0.2616603970527649, test data: 

mean_gamma_deviance failed: (ValueError('Mean Tweedie deviance error with power=2 can only be used on strictly positive y and y_pred.'), 'error while calling <function mean_gamma_deviance 
mean_squared_log_error failed: (ValueError('Mean Squared Logarithmic Error cannot be used when targets contain negative values.'), 'error while calling <function mean_squared_log_error at 0x7
mean_poisson_deviance failed: (ValueError('Mean Tweedie deviance error with power=1 can only be used on non-negative y and strictly positive y_pred.'), 'error while calling <function mean_po
mean_gamma_deviance failed: (ValueError('Mean Tweedie deviance error with power=2 can only be used on strictly positive y and y_pred.'), 'error while calling <function mean_gamma_deviance 
mean_squared_log_error failed: (ValueError('Mean Squared Logarithmic Error cannot be used when targets contain negative values.'), 'error while calling <function mean_squared_log_error at 0x7
mean_poisson_deviance failed: (ValueError('Mean

Training Data,Test Data
,0
r2_score,-1.49
explained_variance_score,-0.00
mean_gamma_deviance,nan
max_error,0.16
mean_absolute_error,0.02
mean_squared_error,0.00
mean_squared_log_error,nan
median_absolute_error,0.02
mean_tweedie_deviance,0.00


In [4]:
import time
time.sleep(0.1)

In [10]:
df.model.predict(fit.model, tail=1)["prediction", "Close"].item()


prediction   feature                                                    \
          Close  0, Close  1, Close  2, Close  3, Close  4, Close  5, Close   
6758  -0.015277 -0.003709  0.004458  0.002266  0.007750  0.002224 -0.001609   
6759  -0.015071 -0.008495 -0.003709  0.004458  0.002266  0.007750  0.002224   
6760  -0.015002 -0.006707 -0.008495 -0.003709  0.004458  0.002266  0.007750   
6761  -0.014901  0.006170 -0.006707 -0.008495 -0.003709  0.004458  0.002266   
6762  -0.014891  0.002023  0.006170 -0.006707 -0.008495 -0.003709  0.004458   

                                              
      6, Close  7, Close  8, Close  9, Close  
6758 -0.003719 -0.000288  0.000738  0.007236  
6759 -0.001609 -0.003719 -0.000288  0.000738  
6760  0.002224 -0.001609 -0.003719 -0.000288  
6761  0.007750  0.002224 -0.001609 -0.003719  
6762  0.002266  0.007750  0.002224 -0.001609